In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
@author: pho
"""
%load_ext autoreload
%autoreload 2
import sys
import importlib
from threading import Thread
import time # for time.sleep
from ipygany import PolyMesh, Scene, IsoColor, WarpByScalar
import pyvista as pv
# from pyvista import _vtk
import pyvistaqt as pvqt
import colorcet as cc # Colormaps:
import numpy as np
import pandas as pd
import h5py
import hdf5storage # conda install hdf5storage
from pathlib import Path
import bqplot.scales
import seaborn as sns
from mpl_toolkits import mplot3d
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from matplotlib.colors import Normalize, to_rgba_array
# import mplcursors
import math # For color map generation
from matplotlib.colors import ListedColormap
from matplotlib.cm import hsv

import ipywidgets as widgets
# from PyQt5 import QtWidgets, uic
from pyvistaqt import QtInteractor, MainWindow
# from pyqt6 import QApplication
from IPython.external.qt_for_kernel import QtGui
from PyQt5.QtWidgets import QApplication

import datetime as dt
import panel as pn # for interactive widgets
pn.extension()

In [2]:
# import PhoPositionalData as pdp
# from PhoPositionalData import load_exported, process_data
from PhoPositionalData.load_exported import *
# from PhoPositionalData.process_data import process_positionalAnalysis_data, gen_2d_histrogram, get_heatmap_color_vectors, process_chunk_equal_poritions_data, extract_spike_timeseries
from PhoPositionalData.process_data import *
from PhoPositionalData.plot_data import *
from PhoPositionalData.plotting.animations import * # make_mp4_from_plotter
from PhoPositionalData.import_data import * # build_spike_positions_list, build_cellID_reverse_lookup_map
from PhoPositionalData.analysis.interactive_placeCell_config import InteractivePlaceCellConfig, VideoOutputModeConfig, PlottingConfig, PlacefieldComputationParameters  # VideoOutputModeConfig, InteractivePlaceCellConfigs
from PhoPositionalData.analysis.interactive_placeCell_config import print_subsession_neuron_differences

def debug_print_spike_counts(session):
    uniques, indicies, inverse_indicies, count_arr = np.unique(active_epoch_session.spikes_df['aclu'].values, return_index=True, return_inverse=True, return_counts=True)
    # count_arr = np.bincount(active_epoch_session.spikes_df['aclu'].values)
    print('active_epoch_session.spikes_df unique aclu values: {}'.format(uniques))
    print('active_epoch_session.spikes_df unique aclu value counts: {}'.format(count_arr))
    print(len(uniques)) # 69 
    uniques, indicies, inverse_indicies, count_arr = np.unique(active_epoch_session.spikes_df['unit_id'].values, return_index=True, return_inverse=True, return_counts=True)
    # count_arr = np.bincount(active_epoch_session.spikes_df['unit_id'].values)
    print('active_epoch_session.spikes_df unique unit_id values: {}'.format(uniques))
    print('active_epoch_session.spikes_df unique unit_id value counts: {}'.format(count_arr))
    
    print(len(uniques)) # 69 
    

In [3]:
# NeuroPy (Diba Lab Python Repo) Loading
try:
    from neuropy import core
    importlib.reload(core)
except ImportError:
    sys.path.append(r'C:\Users\Pho\repos\NeuroPy') # Windows
    # sys.path.append('/home/pho/repo/BapunAnalysis2021/NeuroPy') # Linux
    # sys.path.append(r'/Users/pho/repo/Python Projects/NeuroPy') # MacOS
    print('neuropy module not found, adding directory to sys.path. \n >> Updated sys.path.')
    from neuropy import core
# from neuropy.core.session.dataSession import SessionConfig, DataSessionLoader, DataSession, processDataSssion

from neuropy.core.session.data_session_loader import DataSessionLoader
from neuropy.core.session.dataSession import DataSession
from neuropy.core.epoch import Epoch, NamedTimerange
from neuropy.core import Laps
from neuropy.core import Position
from neuropy.core import FlattenedSpiketrains
from neuropy.core import Neurons
from neuropy.utils.misc import print_seconds_human_readable

## Data must be pre-processed using the MATLAB script located here: 
# R:\data\KDIBA\gor01\one\IIDataMat_Export_ToPython_2021_11_23.m

# From pre-computed .mat files:
## 07: 
# basedir = r'R:\data\KDIBA\gor01\one\2006-6-07_11-26-53'

# ## 08:
basedir = r'R:\data\KDIBA\gor01\one\2006-6-08_14-26-15'

neuropy module not found, adding directory to sys.path. 
 >> Updated sys.path.


In [4]:
from neuropy.analyses import Pf1D, Pf2D, perform_compute_placefields, plot_all_placefields
from neuropy.plotting.spikes import get_neuron_colors

should_force_recompute_placefields = True
should_display_2D_plots = True

def compute_placefields_as_needed(active_session, computation_config=None, active_epoch_placefields1D = None, active_epoch_placefields2D = None, should_force_recompute_placefields=False, should_display_2D_plots=False):
    if computation_config is None:
        computation_config = PlacefieldComputationParameters(speed_thresh=9, grid_bin=2, smooth=0.5)
    active_epoch_placefields1D, active_epoch_placefields2D = perform_compute_placefields(active_session.neurons, active_session.position, computation_config, active_epoch_placefields1D, active_epoch_placefields2D, should_force_recompute_placefields=True)
    # Plot the placefields computed and save them out to files:
    if should_display_2D_plots:
        ax_pf_1D, occupancy_fig, active_pf_2D_figures = plot_all_placefields(active_epoch_placefields1D, active_epoch_placefields2D, active_config)
    else:
        print('skipping 2D placefield plots')
    return active_epoch_placefields1D, active_epoch_placefields2D

## Plotting Colors:
def build_units_colormap(session):
    pf_sort_ind = np.array([int(i) for i in np.arange(len(session.neuron_ids))]) # convert to integer scalar array
    pf_colors = get_neuron_colors(pf_sort_ind) # [4 x n_neurons]: colors are by ascending index ID
    pf_colormap = pf_colors.T # [n_neurons x 4] Make the colormap from the listed colors, used seemingly only by 'runAnalysis_PCAandICA(...)'
    pf_listed_colormap = ListedColormap(pf_colormap)
    return pf_sort_ind, pf_colors, pf_colormap, pf_listed_colormap

def process_by_good_placefields(session, active_config, active_placefields):
    """  Filters the session by the units in active_placefields that have good placefields and return an updated session. Also adds generated colors for each good unit to active_config """
    # Get the cell IDs that have a good place field mapping:
    good_placefield_neuronIDs = np.array(active_placefields.ratemap.neuron_ids) # in order of ascending ID
    print('good_placefield_neuronIDs: {}; ({} good)'.format(good_placefield_neuronIDs, len(good_placefield_neuronIDs)))

    ## Filter by neurons with good placefields only:
    good_placefields_session = active_epoch_session.get_by_id(good_placefield_neuronIDs) # Filter by good placefields only, and this fetch also ensures they're returned in the order of sorted ascending index ([ 2  3  5  7  9 12 18 21 22 23 26 27 29 34 38 45 48 53 57])

    pf_sort_ind, pf_colors, pf_colormap, pf_listed_colormap = build_units_colormap(session)
    active_config.plotting_config.pf_sort_ind = pf_sort_ind
    active_config.plotting_config.pf_colors = pf_colors
    active_config.plotting_config.active_cells_colormap = pf_colormap
    active_config.plotting_config.active_cells_listed_colormap = ListedColormap(active_config.plotting_config.active_cells_colormap)
    
    return good_placefields_session, active_config, good_placefield_neuronIDs


## For building the configs used to filter the session by epoch: 
def build_configs(session_config, active_epoch, active_subplots_shape = (1,1)):
    ## Get the config corresponding to this epoch/session settings:
    active_config = InteractivePlaceCellConfig(active_session_config=session_config, active_epochs=active_epoch, video_output_config=None, plotting_config=None) # '3|1    

    active_config.video_output_config = VideoOutputModeConfig(active_frame_range=np.arange(100.0, 120.0), 
                                                              video_output_parent_dir=Path('output', active_sess_config.session_name, active_epoch.name),
                                                              active_is_video_output_mode=False)
    active_config.plotting_config = PlottingConfig(output_subplots_shape=active_subplots_shape,
                                                   output_parent_dir=Path('output', active_sess_config.session_name, active_epoch.name)
                                                  )
    # Make the directories:
    active_config.plotting_config.active_output_parent_dir.mkdir(parents=True, exist_ok=True) # makes the directory if it isn't already there
    return active_config


In [34]:
# KDiba Old Format:
sess = DataSessionLoader.kdiba_old_format_session(basedir)

	 basepath: R:\data\KDIBA\gor01\one\2006-6-08_14-26-15
	 session_name: 2006-6-08_14-26-15
Loading matlab import file: R:\data\KDIBA\gor01\one\2006-6-08_14-26-15\2006-6-08_14-26-15.epochs_info.mat...
done.
Loading matlab import file: R:\data\KDIBA\gor01\one\2006-6-08_14-26-15\2006-6-08_14-26-15.position_info.mat...
done.
Loading matlab import file: R:\data\KDIBA\gor01\one\2006-6-08_14-26-15\2006-6-08_14-26-15.spikeII.mat...
done.


C:\Users\Pho\repos\NeuroPy\neuropy\core\session\data_session_loader.py:679: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  spikes_df.lap_maze[0:split_index] = 1
C:\Users\Pho\repos\NeuroPy\neuropy\core\session\data_session_loader.py:680: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  spikes_df.lap_maze[split_index:] = 2 # maze 2
C:\Users\Pho\repos\NeuroPy\neuropy\core\session\data_session_loader.py:681: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versu

setting laps object.
computing linear positions for all active epochs for session...
Saving updated position results to R:\data\KDIBA\gor01\one\2006-6-08_14-26-15\2006-6-08_14-26-15.position.npy...
2006-6-08_14-26-15.position.npy saved
done.

session.position linear positions computed!


In [35]:
active_sess_config = sess.config
session_name = sess.name
# Setup the placefield computation config
# active_config.computation_config = PlacefieldComputationParameters(speed_thresh=3, grid_bin=2, smooth=2)
sess.epochs.to_dataframe()

,start,stop,label,duration
0,0.00000,1211.558080,maze1,1211.558080
1,1211.55808,2093.897857,maze2,882.339777


In [36]:
# sess.spikes_df.size # (1014937, 18)
# np.sum(sess.neurons.n_spikes) # 1014937
print('session dataframe spikes: {}\n session.neurons.n_spikes summed: {}\n'.format(sess.spikes_df.shape, np.sum(sess.neurons.n_spikes)))

session dataframe spikes: (1014937, 20)
 session.neurons.n_spikes summed: 1014937



## Debugging

In [21]:
# debug_print_spike_counts(sess)

# sess.spikes_df unique aclu values: [ 2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25
#  26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49
#  50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65]
# sess.spikes_df unique aclu value counts: [    0     0  1707  1359   554  3066 91337  2707   294  2991   913  1278
#   1221  5475  2880 36760 48449   523  2595  3025 34729  3958  5263   790
#    663  1070  1078  2954  4044  2581 73951  4871 13444   476  1234 89854
#   5822 14961  2682  1184 13575 83972   795 22769  1559  8307  2098   438
#   3476 28370 43854  1355  2552  2875 59011  1440 75786  3656   875 17727
#  77247   550   408  2627  1734  2799]

# sess.spikes_df unique unit_id values: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
#  24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
#  48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63]
# sess.spikes_df unique unit_id value counts: [ 1707  1359   554  3066 91337  2707   294  2991   913  1278  1221  5475
#   2880 36760 48449   523  2595  3025 34729  3958  5263   790   663  1070
#   1078  2954  4044  2581 73951  4871 13444   476  1234 89854  5822 14961
#   2682  1184 13575 83972   795 22769  1559  8307  2098   438  3476 28370
#  43854  1355  2552  2875 59011  1440 75786  3656   875 17727 77247   550
#    408  2627  1734  2799]

# sess.neurons.spiketrains
# sess.neurons.n_spikes

## Note the unit_id seems to have valid spikes for all of them, while the aclu does not!
sess.spikes_df # 1014937

# np.where(np.logical_and((sess.spikes_df.maze_relative_lap == 1), (sess.spikes_df.lap_maze == 2)))
# sess.spikes_df.lap.values[np.logical_and((sess.spikes_df.maze_relative_lap == 1), (sess.spikes_df.lap_maze == 2))]


# test_df = sess.spikes_df
# test_df

(array([616655], dtype=int64),)

In [ ]:
# len(sess.neurons.n_spikes) # 64 items long
# sess.neurons.n_spikes
# array([ 1707,  1359,   554,  3066, 91337,  2707,   294,  2991,   913,
#         1278,  1221,  5475,  2880, 36760, 48449,   523,  2595,  3025,
#        34729,  3958,  5263,   790,   663,  1070,  1078,  2954,  4044,
#         2581, 73951,  4871, 13444,   476,  1234, 89854,  5822, 14961,
#         2682,  1184, 13575, 83972,   795, 22769,  1559,  8307,  2098,
#          438,  3476, 28370, 43854,  1355,  2552,  2875, 59011,  1440,
#        75786,  3656,   875, 17727, 77247,   550,   408,  2627,  1734,
#         2799])

## Laps: Create version of Session that only includes activity during the laps

In [30]:
sess.laps.lap_id # 1-80

# np.unique(sess.spikes_df.lap)

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51,
       52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68,
       69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80], dtype=int64)

In [38]:
lap_specific_session = sess.filtered_by_laps(lap_indicies=np.arange(3, 81))
print(lap_specific_session.neurons.n_spikes)

In [48]:
# # lap_specific_session.spikes_df
lap_specific_subsessions, lap_specific_dataframes, lap_spike_indicies, lap_spike_t_seconds = Laps.build_lap_specific_lists(sess, include_empty_lists=True)
# sess.neurons.n_spikes

Constraining to epoch with times (start: 28.666600000000003, end: 4048.9707000000003)
Constraining to epoch with times (start: 109.94120000000001, end: 127.7231)
Constraining to epoch with times (start: 442.9429, end: 467.8911)
Constraining to epoch with times (start: 525.0678, end: 541.2292)
Constraining to epoch with times (start: 716.5310000000001, end: 776.3871)
Constraining to epoch with times (start: 836.2068, end: 850.6622)
Constraining to epoch with times (start: 959.1822000000001, end: 974.2764000000001)
Constraining to epoch with times (start: 1021.8599, end: 1035.771)
Constraining to epoch with times (start: 1615.1453000000001, end: 1640.0033)
Constraining to epoch with times (start: 1665.8343, end: 1680.9245)
Constraining to epoch with times (start: 1732.5332, end: 1754.8831)
Constraining to epoch with times (start: 1823.8762000000002, end: 1836.7985)
Constraining to epoch with times (start: 1901.6403, end: 1921.1926)
Constraining to epoch with times (start: 1949.0218, end:

In [50]:
# lap_specific_subsessions, lap_specific_dataframes, lap_spike_indicies, lap_spike_t_seconds = Laps.build_lap_specific_lists(sess, include_empty_lists=True)
lap_indices = np.arange(4, 79)
if lap_indices is None:
    lap_indices = np.arange(1, len(lap_specific_subsessions)) # all laps by default

print('filtering by laps: {}'.format(lap_indices)) 
active_lap_specific_subsessions = [lap_specific_subsessions[i] for i in lap_indices] # filter by the desired number of laps 
## Effectively build the new session using only the lap-specific spiketimes:
lap_specific_session = DataSession.concat(active_lap_specific_subsessions)

filtering by laps: [ 4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27
 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51
 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75
 76 77 78]
!! WARNING: Session.concat(...) is not yet fully implemented, meaning the returned session is not fully valid. Continue with caution.


In [33]:
len(lap_specific_subsessions) # 80
sess.laps.lap_start_stop_flat_idx

array([[  5902., 616654.],
       [ 19997.,  24024.],
       [ 69332.,  74904.],
       [ 83437.,  87077.],
       [110178., 121020.],
       [130178., 133426.],
       [151439., 154900.],
       [161637., 164631.],
       [237338., 242785.],
       [246320., 249661.],
       [257345., 262028.],
       [271252., 274188.],
       [283338., 287617.],
       [291221., 294602.],
       [300854., 305618.],
       [315173., 321830.],
       [334006., 337261.],
       [340252., 343644.],
       [349941., 355254.],
       [359228., 365760.],
       [369715., 374067.],
       [377128., 380411.],
       [384921., 390892.],
       [398722., 402036.],
       [408858., 413187.],
       [420226., 423145.],
       [447896., 453724.],
       [458076., 461317.],
       [474023., 479216.],
       [484643., 487689.],
       [490852., 499415.],
       [504498., 507865.],
       [513772., 518362.],
       [521745., 525010.],
       [528403., 535106.],
       [540908., 543908.],
       [549996., 553886.],
 

In [12]:
spiketrains_list = list()
num_neurons = lap_specific_subsessions[0].neurons.n_neurons
objList = [aSession.neurons for aSession in lap_specific_subsessions]

nSpikesList = [aSession.neurons.n_spikes for aSession in lap_specific_subsessions]
# np.shape(objList[0].spiketrains) # (108,)
# neuron_idx = 0
# test = [obj.spiketrains[neuron_idx] for obj in objList]
# np.shape(test[0]) # (219,) not correct

spiketrains_list = objList[0].spiketrains
for neuron_idx in np.arange(num_neurons):
    for obj_idx in np.arange(1, len(objList)):
        # spiketrains_list[neuron_idx].append(objList[obj_idx].spiketrains[neuron_idx])
        spiketrains_list[neuron_idx] = np.append(spiketrains_list[neuron_idx], objList[obj_idx].spiketrains[neuron_idx])
spiketrains_list

array([array([  32.8037,   33.1626,   33.1946, ..., 3903.2451, 3996.1426,
              4500.2168])                                                ,
       array([  37.5993,  767.5956, 1632.1545, ..., 6727.8332, 6727.8555,
              6727.934 ])                                                ,
       array([2444.4685, 3014.7803, 3907.1512, ..., 6677.2467, 6725.8469,
              6726.2247])                                                ,
       array([  29.0388,   37.6272,   40.5232, ..., 6719.4075, 6722.0643,
              6722.3732])                                                ,
       array([  28.8534,   29.0705,   29.1722, ..., 6728.1872, 6728.5898,
              6729.7497])                                                ,
       array([  98.2777,   98.3084,  133.803 , ..., 6686.1221, 6718.9951,
              6719.0194])                                                ,
       array([ 120.5338,  120.566 ,  120.9416, ..., 6501.9797, 6620.1819,
              6620.1984])       

In [54]:
# for neuron_idx in np.arange(num_neurons):
#     curr_neuron_spiketrains_list = np.concatenate([obj.spiketrains[neuron_idx] for obj in objList], axis=0)
#     spiketrains_list.append(curr_neuron_spiketrains_list)
    
# # curr_neuron_spiketrains_list
# spiketrains_list

# Filter Session by Epoch:

In [51]:
# active_epoch = sess.epochs.get_named_timerange('maze1')
# active_epoch = sess.epochs.get_named_timerange('maze2')
active_epoch = NamedTimerange(name='maze', start_end_times=[sess.epochs['maze1'][0], sess.epochs['maze2'][1]])

active_subplots_shape = (1,1) # Single subplot
# active_subplots_shape = '1|2' # 1 subplot on left, two on right                                                   
active_config = build_configs(active_sess_config, active_epoch, active_subplots_shape = active_subplots_shape)                                  
active_config.computation_config = PlacefieldComputationParameters(speed_thresh=9, grid_bin=2, smooth=0.5)

## All Spikes:
active_epoch_session = sess.filtered_by_neuron_type('pyramidal').filtered_by_epoch(active_epoch)
print_subsession_neuron_differences(sess.neurons, active_epoch_session.neurons)
## Lap_specific Spikes Only:
active_lap_specific_epoch_session = lap_specific_session.filtered_by_neuron_type('pyramidal').filtered_by_epoch(active_epoch)
print_subsession_neuron_differences(lap_specific_session.neurons, active_lap_specific_epoch_session.neurons)

Constraining to units with type: pyramidal
Constraining to epoch with times (start: 0.0, end: 2093.8978568242164)
34301/1014937 total spikes spanning 70/108 units remain in subsession
Constraining to units with type: pyramidal
Constraining to epoch with times (start: 0.0, end: 2093.8978568242164)
10626/541639 total spikes spanning 70/108 units remain in subsession


In [52]:
print(sess.neurons.n_spikes)
print(lap_specific_session.neurons.n_spikes)

[  235   902   521  3465 60228  2845   586  3962  1493  1334 48712   280
  1843   426  1575  1875 45467   653 33885   396  1459   527   738   933
   248   911   728 16975 79907  1429  4175  1035  1509  3416  1519  2585
  2224  3315 38546  1840 26964 17665 18591  2292  1903 92936  7552  1990
  1783  1069  2004  1806  1055  2006  1668  2423   939  2841  2642  3351
  5160  2659  1499 65675  3247  1511  1357  1679  1137  1440  2901   447
  1252  4626  1185 72724  3365  1154 74260  1238   813  2871  1977  1408
  3434   396  2839  1817  2562  5025  1517  1355 38643  3113   847   810
  4654  1466  1604  3435  1870 32017 32190    63 42001  2784  6489   239]
[  192   987   426  1575 25936  1890   271   322  1784   663 26457    26
  1062   618   215  1203 30953    14 12115   242   878   462  1282  1131
   335   150    22  8783 42988  1333   570   910   982   332    62   108
   636  1696 15806  1003 13852  8638 10386  1160   956 48286  2651  1785
  1750   403   940   851   414  1653  1009   737  

## Compute Placefields if needed:

In [ ]:
## ICA and PCA Analysis
should_show_2D_ICA_plots = False
from PhoPositionalData.analysis.neuronal_dimensionality_reduction import runAnalysis_PCAandICA
active_session_ensembles, template, zsc_template, pca_data = runAnalysis_PCAandICA(active_epoch_session.neurons, bin_size=0.250, frate_thresh=0.1, should_plot=should_show_2D_ICA_plots, active_cells_colormap=active_config.plotting_config.active_cells_colormap)

In [ ]:
try: active_epoch_placefields1D
except NameError: active_epoch_placefields1D = None # Checks variable active_epoch_placefields's existance, and sets its value to None if it doesn't exist so it can be checked in the next step
try: active_epoch_placefields2D
except NameError: active_epoch_placefields2D = None # Checks variable active_epoch_placefields's existance, and sets its value to None if it doesn't exist so it can be checked in the next step

active_epoch_placefields1D, active_epoch_placefields2D = compute_placefields_as_needed(active_epoch_session, active_config.computation_config, active_epoch_placefields1D, active_epoch_placefields2D, should_force_recompute_placefields=should_force_recompute_placefields, should_display_2D_plots=should_display_2D_plots)
# Focus on the 2D placefields:
active_epoch_placefields = active_epoch_placefields2D
# Get the updated session using the units that have good placefields
active_epoch_session, active_config, good_placefield_neuronIDs = process_by_good_placefields(active_epoch_session, active_config, active_epoch_placefields)

## Compute the placefields for lap-only filtered Session:

In [ ]:
try: active_lap_only_placefields1D
except NameError: active_lap_only_placefields1D = None # Checks variable active_epoch_placefields's existance, and sets its value to None if it doesn't exist so it can be checked in the next step
try: active_lap_only_placefields2D
except NameError: active_lap_only_placefields2D = None # Checks variable active_epoch_placefields's existance, and sets its value to None if it doesn't exist so it can be checked in the next step

active_lap_only_placefields1D, active_lap_only_placefields2D = compute_placefields_as_needed(lap_specific_session, lap_specific_config.computation_config, active_lap_only_placefields1D, active_lap_only_placefields2D, should_force_recompute_placefields=should_force_recompute_placefields, should_display_2D_plots=should_display_2D_plots)
# Focus on the 2D placefields:
active_lap_only_placefields = active_lap_only_placefields2D
# Get the updated session using the units that have good placefields
lap_specific_session, lap_specific_config, lap_specific_good_placefield_neuronIDs = process_by_good_placefields(lap_specific_session, lap_specific_config, active_lap_only_placefields)

## Debug

In [ ]:
debug_print_spike_counts(active_epoch_session)
# active_epoch_session.spikes_df unique aclu values: [ 2  3  4  5  7  8  9 10 11 12 14 17 18 21 22 23 24 25 26 27 28 29 33 34
#  38 39 42 44 45 46 47 48 53 55 57 58 61 62 63 64]
# active_epoch_session.spikes_df unique aclu value counts: [   0    0  513  473   33  630    0  130  100  631   70  150  349    0
#    52    0    0  308  952    0    0 1436  898  342  223   89  109 1261
#   684  916    0    0    0  137  293    0    0    0  687  394    0    0
#   225    0  495 2123  316  111  396    0    0    0    0  458    0  576
#     0  927  382    0    0   89   90   55  370]
# active_epoch_session.spikes_df unique unit_id values: [ 0  1  2  3  5  6  7  8  9 10 12 15 16 19 20 21 22 23 24 25 26 27 31 32
#  36 37 40 42 43 44 45 46 51 53 55 56 59 60 61 62]
# active_epoch_session.spikes_df unique unit_id value counts: [ 513  473   33  630    0  130  100  631   70  150  349    0   52    0
#     0  308  952    0    0 1436  898  342  223   89  109 1261  684  916
#     0    0    0  137  293    0    0    0  687  394    0    0  225    0
#   495 2123  316  111  396    0    0    0    0  458    0  576    0  927
#   382    0    0   89   90   55  370]


# print(active_epoch_session.epochs.to_dataframe())
# print(sess.spikes_df) # max is in 4.2305 x 10^4
# print(sess.position.to_dataframe()) # max is in 4.2305 x 10^4 ## NOTE: The only place that I need to be careful with indexing is with sess.position properties, as these appear to be represented in terms of the number of 60Hz samples instead of in seconds like the Neurons and other classes.
# print(sess.flattened_spiketrains.spikes_df) # t_seconds is in the same units as the position dataframe, which is NOT seconds, and 't' is a factor of 1x10^4 larger than 't_seconds'
# # Get only the rows with a lap != -1:
# # spikes_df = spikes_df[(spikes_df.lap != -1)] # 229887 rows × 13 columns
# # spikes_df # 932598 rows × 14 columns

In [ ]:
print('pix2cm: {}'.format(DataSessionLoader.pix2cm))

In [ ]:
active_epoch_session.spikes_df

In [ ]:
debug_print_spike_counts(active_epoch_session) # 46 neurons after filtering, much better

# count_arr = np.bincount(active_epoch_session.spikes_df['unit_id'])
# count_arr
valid_indicies = np.nonzero(active_epoch_session.spikes_df['unit_id'].values)
valid_indicies
# count_arr = np.bincount(active_epoch_session.spikes_df['aclu'])
# count_arr

# len(active_epoch_session.spikes_df['unit_id'].values) # 17766
active_epoch_session.neurons.n_spikes

# active_epoch_session.neurons.n_spikes # array([ 513,  473,   33,  630,  130,  631,  150,  349,  308,  952, 1436,
#         # 898,  342,  223, 1261,  684,  916,  137,  293,  687,  394,  225,
#         # 495, 2123,  316,  111,  396,  458,  576,  927,  382])
        
# array([   0,    0,  513,  473,   33,  630,    0,  130,    0,  631,    0,
#         150,  349,    0,    0,    0,    0,  308,  952,    0,    0, 1436,
#         898,  342,  223,    0,    0, 1261,  684,  916,    0,    0,    0,
#         137,  293,    0,    0,    0,  687,  394,    0,    0,  225,    0,
#         495, 2123,  316,  111,  396,    0,    0,    0,    0,  458,    0,
#         576,    0,  927,  382], dtype=int64)

In [ ]:
active_epoch_session.spikes_df

## Main Spike/Placemap plotting:

# Plot Placemaps on 3D Plot

## Tuning Curves 3D Plot:

In [ ]:
from PhoGui.InteractivePlotter.InteractivePlaceCellTuningCurvesDataExplorer import InteractivePlaceCellTuningCurvesDataExplorer
try: pActiveTuningCurvesPlotter
except NameError: pActiveTuningCurvesPlotter = None # Checks variable p's existance, and sets its value to None if it doesn't exist so it can be checked in the next step
ipcDataExplorer = InteractivePlaceCellTuningCurvesDataExplorer(active_config, active_epoch_session, active_epoch_placefields, pf_colors, extant_plotter=pActiveTuningCurvesPlotter)
pActiveTuningCurvesPlotter = ipcDataExplorer.plot(pActiveTuningCurvesPlotter) # [2, 17449]

## Debugging Index Issues:

In [ ]:
# def plot_placefields2D(pTuningCurves, active_placefields, pf_colors: np.ndarray, zScalingFactor=10.0, show_legend=False):

#     # .threshold().elevation()

#     curr_tuning_curves = active_placefields.ratemap.normalized_tuning_curves
#     # curr_tuning_curves[curr_tuning_curves < 0.1] = np.nan
#     curr_tuning_curves = curr_tuning_curves * zScalingFactor

#     num_curr_tuning_curves = len(curr_tuning_curves)
#     # Get the cell IDs that have a good place field mapping:
#     good_placefield_neuronIDs = np.array(active_placefields.ratemap.neuron_ids) # in order of ascending ID
#     tuningCurvePlot_x, tuningCurvePlot_y = np.meshgrid(active_placefields.ratemap.xbin_centers, active_placefields.ratemap.ybin_centers)

#     pdata_currActiveNeuronTuningCurve = pv.StructuredGrid(tuningCurvePlot_x, tuningCurvePlot_y, curr_active_neuron_tuning_Curve)
#     pdata_currActiveNeuronTuningCurve["Elevation"] = curr_active_neuron_tuning_Curve.ravel(order="F")
        
#     return pTuningCurves

# plot_placefields2D(pTuningCurves, active_epoch_placefields

# ipcDataExplorer.plots['spikes_pf_active']['cellID']

# only_active_colormap = ipcDataExplorer.active_config.plotting_config.active_cells_listed_colormap.copy()
# only_active_colormap
test_full_pc_data = ipcDataExplorer.plots_data['spikes_pf_active']['historical_spikes_pc']

# test_subset = test_full_pc_data.GetCellGhostArray()
# test_subset = test_full_pc_data.GetCellData() # vtkmodules.vtkCommonDataModel.vtkCellData
# test_subset = test_full_pc_data[test_full_pc_data['cellID'] == 55] # vtkmodules.vtkCommonDataModel.vtkCellData

# test_subset = test_full_pc_data.GetGhostArray() # [test_full_pc_data['cellID'] == 55] # vtkmodules.vtkCommonDataModel.vtkCellData


unique_ids = np.unique(test_full_pc_data['cellID']) # array([ 0,  1,  2,  3,  5,  7,  9, 10, 15, 16, 19, 20, 21, 22, 25, 26, 27, 31, 32, 36, 37, 40, 42, 43, 44, 45, 46, 51, 53, 55, 56])
# count_arr = np.bincount(test_full_pc_data['cellID'])
count_arr
# array([ 3591,  3311,   231,  4410,     0,   910,     0,  4417,     0,
#         1050,  2443,     0,     0,     0,     0,  2156,  6664,     0,
#            0, 10052,  6286,  2394,  1561,     0,     0,  8827,  4788,
#         6412,     0,     0,     0,   959,  2051,     0,     0,     0,
#         4809,  2758,     0,     0,  1575,     0,  3465, 14861,  2212,
#          777,  2772,     0,     0,     0,     0,  3206,     0,  4032,
#            0,  6489,  2674], dtype=int64)

# good_placefield_neuronIDs: [  5   8  10  13  14  16  19  21  23  25  28  31  32  33  36  37  41  49
#   52  53  54  55  57  59  60  61  62  63  64  66  68  69  74  75  76  78
#   83  86  88  89  90  92  96  98 105 108]; (46 good)

# 1, 4, 6, 7, 8, 11, 18, 19, 21 ## CONCLUSION: the ones that work are indeed the non-zero entries, but the checkboxes work in reverse order to this array that's printed. Meaning you start at the end and work back.
# len(count_arr) # 107
print(unique_ids) # [  3   6   8  11  12  14  17  19  21  23  26  29  30  31  34  35  39  47
  # 50  51  52  53  55  57  58  59  60  61  62  64  66  67  72  73  74  76
  # 81  84  86  87  88  90  94  96 103 106]


# uniques, indicies, inverse_indicies, count_arr = np.unique(active_epoch_session.spikes_df['aclu'].values, return_index=True, return_inverse=True, return_counts=True)
# # count_arr = np.bincount(active_epoch_session.spikes_df['aclu'].values)
# print('active_epoch_session.spikes_df unique aclu values: {}'.format(uniques))
# print('active_epoch_session.spikes_df unique aclu value counts: {}'.format(count_arr))
# print(len(uniques)) # 46 
# uniques, indicies, inverse_indicies, count_arr = np.unique(active_epoch_session.spikes_df['unit_id'].values, return_index=True, return_inverse=True, return_counts=True)
# # count_arr = np.bincount(active_epoch_session.spikes_df['unit_id'].values)
# print('active_epoch_session.spikes_df unique unit_id values: {}'.format(uniques))
# print('active_epoch_session.spikes_df unique unit_id value counts: {}'.format(count_arr))
# print(len(uniques)) # 46 

In [ ]:
# pActiveTuningCurvesPlotter.export_obj('export.obj')
# pActiveTuningCurvesPlotter.export_gltf('export.gltf')

ipcDataExplorer.update_placefield_spike_visibility([38], True) # seems to work

In [ ]:
ipcDataExplorer.update_placefield_spike_visibility([53], True) # seems to work

In [ ]:
ipcDataExplorer.update_placefield_spike_visibility([44, 53], True) # seems to work

# test_subset
# test_subset

In [ ]:
ipcDataExplorer.update_placefield_spike_visibility([53, 44], False) # seems to work

In [ ]:
ipcDataExplorer.gui['tuningCurveSpikeVisibilityCallbacks'][1](False)
# ipcDataExplorer.get_cell_index([2, 3])

In [ ]:
mesh = ipcDataExplorer.plots_data['spikes_pf_active']['historical_spikes_pc'].cast_to_unstructured_grid()
# mesh.n_cells
mesh_unique_ids = np.unique(mesh['cellID'])
print('n_cells: {}'.format(mesh.n_cells)) # 122143
print('cellIDs of mesh: {}'.format(mesh_unique_ids))
# np.unique(mesh['cellID']) # array([ 0,  1,  2,  3,  5,  7,  9, 10, 15, 16, 19, 20, 21, 22, 25, 26, 27, 31, 32, 36, 37, 40, 42, 43, 44, 45, 46, 51, 53, 55, 56])
# ipcDataExplorer.hide_placefield_spikes([0,1,2,3], should_invert=True)
# ipcDataExplorer.hide_placefield_spikes([23,2,34], should_invert=True)

# ipcDataExplorer.hide_placefield_spikes([34], should_invert=True)
ipcDataExplorer.hide_placefield_spikes([38], should_invert=False)

In [ ]:
# the list of spike times that occured for this cell:
i = 2
active_epoch_session.neurons.neuron_ids[i]
active_epoch_session.neurons.spiketrains[i]

# spikes_df = FlattenedSpiketrains.build_spike_dataframe(sess)


## Interactive 3D Spike and Behavior Browser: 

In [ ]:
import PhoGui
from PhoGui.InteractivePlotter.PhoInteractivePlotter import PhoInteractivePlotter
from PhoGui.InteractivePlotter.shared_helpers import InteractivePyvistaPlotterBuildIfNeededMixin
from PhoGui.InteractivePlotter.InteractivePlaceCellDataExplorer import InteractivePlaceCellDataExplorer

active_config.plotting_config.show_legend = True

try: pActiveInteractivePlaceSpikesPlotter
except NameError: pActiveInteractivePlaceSpikesPlotter = None # Checks variable p's existance, and sets its value to None if it doesn't exist so it can be checked in the next step
ipspikesDataExplorer = InteractivePlaceCellDataExplorer(active_config, active_epoch_session, extant_plotter=pActiveInteractivePlaceSpikesPlotter)
pActiveInteractivePlaceSpikesPlotter = ipspikesDataExplorer.plot(pActivePlotter=pActiveInteractivePlaceSpikesPlotter)

# Programmatically change the animal position trail:

In [ ]:
def plot_lap_trajectory_path(ipspikesDataExplorer, curr_lap_position_traces):
    num_lap_samples = np.shape(curr_lap_position_traces)[1]
    lap_fixed_z = np.full_like(curr_lap_position_traces[0,:], 0.9)
    plot_name = 'lap_location_trail'
    # fade_values = ipspikesDataExplorer.params.active_trail_opacity_values
    # size_values = ipspikesDataExplorer.params.active_trail_size_values
    trail_fade_values = None
    size_values = None
    # trail_fade_values = np.linspace(0.0, 0.6, num_lap_samples)
    size_values = np.linspace(0.2, 0.6, num_lap_samples) # fade from a scale of 0.2 to 0.6
    ipspikesDataExplorer.perform_plot_location_trail(plot_name, curr_lap_position_traces[0,:], curr_lap_position_traces[1,:], lap_fixed_z,
                                                 trail_fade_values=trail_fade_values, trail_point_size_values=size_values,
                                                 render=True, color='red')

# def hide_lap_trajectory_path(ipspikesDataExplorer):
    



    
## ACTIVE: Laps and Spikes:

# def visually_indicate_current_lap(
curr_lap_id = 0

curr_lap_t_start, curr_lap_t_stop = get_lap_times(curr_lap_id)
curr_lap_subsession = lap_specific_subsessions[curr_lap_id]
curr_lap_dataframe = lap_specific_dataframes[curr_lap_id] # the subset dataframe for this lap
curr_lap_spike_t_seconds = curr_lap_dataframe.t_seconds
curr_lap_spike_indicies = lap_spike_indicies[curr_lap_id] # all spike indicies that occured within the lap
curr_lap_position_traces = curr_lap_subsession.position.traces # the animal positions that were traversed during this lap

curr_lap_num_spikes = len(curr_lap_spike_indicies)
print('lap[{}]: ({}, {}): {} spikes.'.format(curr_lap_id, curr_lap_t_start, curr_lap_t_stop, curr_lap_num_spikes))

test_flat_times = active_epoch_session.flattened_spiketrains.flattened_spike_times
print(test_flat_times)
    
plot_lap_trajectory_path(ipspikesDataExplorer, curr_lap_position_traces)
# curr_lap_t_start
## TODO: enable showing/hiding the spikes for this data range programmatically in InteractivePlaceCellDataExplorer. 
# ipspikesDataExplorer.
# curr_lap_spike_t_seconds
# curr_lap_spike_indicies
# active_epoch_session.flattened_spiketrains.spikes_df.t_seconds.values[curr_lap_spike_indicies] # index 145937 is out of bounds for axis 0 with size 19647

In [ ]:
renderWindow.SetWindowName('Test')
# ipspikesDataExplorer.on_programmatic_data_update(active_included_all_historical_indicies=curr_lap_spike_indicies, active_included_recent_only_indicies=curr_lap_spike_indicies) # index 145937 is out of bounds for axis 0 with size 19647
# ipspikesDataExplorer.on_programmatic_data_update(active_included_all_historical_indicies=curr_lap_spike_t_seconds, active_included_recent_only_indicies=curr_lap_spike_t_seconds) # 
# ipspikesDataExplorer.on_programmatic_data_update(active_included_all_historical_indicies=curr_lap_spike_t_seconds.values, active_included_recent_only_indicies=curr_lap_spike_t_seconds.values) # 

In [ ]:
# ipspikesDataExplorer.spikes_main_historical.SetVisibility(0)

# ipspikesDataExplorer.spikes_main_historical.SetVisibility(1)
# bool(ipspikesDataExplorer.spikes_main_historical.GetVisibility())

ipspikesDataExplorer.toggle_plot_visibility('spikes_main_historical')
# toggle_visibility(ipspikesDataExplorer.spikes_main_historical)

In [ ]:
ipspikesDataExplorer.toggle_plot_visibility('lap_location_trail')

In [ ]:
# ipspikesDataExplorer.spikes_main_recent_only.SetVisibility(True)
toggle_visibility(ipspikesDataExplorer.spikes_main_recent_only)

In [ ]:
# ipspikesDataExplorer.animal_location_trail.SetVisibility(0)
toggle_visibility(ipspikesDataExplorer.animal_location_trail)

In [ ]:
# ipspikesDataExplorer.animal_location_trail.SetVisibility(0)
toggle_visibility(ipspikesDataExplorer.animal_current_location_point)

# Programmatically change the animals location point indicator:

In [ ]:
# .GetPosition() returns the actor's position, which is usually (0.0, 0.0, 0.0), and does not relate to the actor's data points
# curr_animal_point = ipspikesDataExplorer.animal_location_trail.GetPosition()
# curr_animal_point = ipspikesDataExplorer.animal_location_trail.GetXRange() # (44.23604202270508, 245.9059600830078)
# print(curr_animal_point)
# curr_animal_point = ipspikesDataExplorer.animal_location_trail.GetYRange() # (135.27638244628906, 145.448974609375)
# print(curr_animal_point)
# curr_animal_point = ipspikesDataExplorer.animal_location_trail.GetZRange() # (1.100000023841858, 1.100000023841858)

curr_animal_point = np.array(ipspikesDataExplorer.animal_location_trail.GetCenter()) # (206.27755737304688, 140.15452575683594, 1.100000023841858)
print('curr_animal_point: {}'.format(curr_animal_point))
curr_animal_point # curr_animal_poin

# curr_animal_point = np.array([0, 0, 0])

# curr_animal_point = np.column_stack((self.x[active_window_sample_indicies], self.y[active_window_sample_indicies], self.z_fixed))
# ipspikesDataExplorer.on_programmatic_data_update(curr_animal_point=curr_animal_point)

curr_debug_point = np.array(ipspikesDataExplorer.animal_location_trail.GetCenter()) # (206.27755737304688, 140.15452575683594, 1.100000023841858)
ipspikesDataExplorer.perform_plot_location_point('debug_point_plot', curr_animal_point, color='r')

curr_animal_point = np.array(ipspikesDataExplorer.animal_location_trail.GetCenter()) # not updated
print('new curr_animal_point: {}'.format(curr_animal_point))

In [ ]:
# ipspikesDataExplorer.flattened_spike_positions_list

active_epoch_session.flattened_spiketrains.time_slice(curr_lap_spike_t_seconds.values[0], curr_lap_spike_t_seconds.values[-1]).spikes_df

In [ ]:
curr_lap_dataframe

In [ ]:
active_epoch_session.time_slice